In [22]:
import pandas as pd
import numpy as np
import datetime 
import json

In [23]:
def is_date(strdate):
    '''判断是否是一个有效的日期字符串'''
    try:
        if ":" in strdate:
            datetime.datetime.strptime(strdate, "%Y-%m-%d %H:%M:%S")
        else:
            datetime.datetime.strptime(strdate, "%Y-%m-%d")
        return True
    except:
        return False

In [24]:
c_t = "rdfs:Class"
p_t = "rdf:Property"
label_t = "rdfs:label"
c_xml = "http://www.openkg.cn/COVID-19/epidemiology/class/"
p_xml = "http://www.openkg.cn/COVID-19/epidemiology/property/"
r_xml = "http://www.openkg.cn/COVID-19/epidemiology/resource/"
# label标签 ："rdfs:label"
path = r"data/流行病.json"

In [25]:
'''创建Map'''
class_map = {}
property_map = {}

In [16]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)['@graph']
    for G in f:
        if G['@id'].find(c_xml) != -1:
            class_map[G['@id'].split(c_xml)[-1]] = G[label_t]
        elif G['@type'] == p_t:
            if G['@id'].find(p_xml) != -1:
                property_map[G['@id'].split(p_xml)[-1]] = G[label_t]
            else:
                property_map[G['@id'].replace(':','')] = G[label_t]
        else:
            pass

In [26]:
entity_id = []
entity_value = []
entity_label = []
rel_from = []
rel_to = []
rel_type = []
dateC = 1
stringC = 1

entity = pd.DataFrame()
rel = pd.DataFrame()

In [18]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)
    for G in f:
        if G['@id'].find(r_xml) !=-1 :
            r_id = G['@id'].split(r_xml)[-1] # main resource的 RID
            entity_id.append(r_id)
            entity_value.append(G[label_t])
            entity_label.append(class_map[ G['@type'].split(c_xml)[-1] ])
            '''Rel'''
            for k in G.keys():
                if k.find(p_xml)!=-1:
                    # 一个PID可能有多个对应的Resource 或者是单独的String/Date
                    ktail = k.split(p_xml)[-1] #截去xml 保留P+ID
                    # 如果是string则是@value 而若是resource_map则是@id
                    if '@id' in G[k][-1]:
                        if G[k][-1]['@id'].find(r_xml)!=-1: 
                            for v in G[k]:
                                rel_from.append(r_id)
                                rel_type.append(property_map[ktail]) # 创建main resource和property
                                rel_to.append(v['@id'].split(r_xml)[-1])
                    elif '@value' in G[k][-1]:
                        rel_from.append(r_id)
                        rel_type.append(property_map[ktail]) # 创建main resource和property
                        
                        if is_date(G[k][-1]['@value']) == True: # 若是date字符串则 需要创建entity 并设置RID
                            dr_id = 'dR'+ str(dateC) # Character_Date dR+ID
                            dateC += 1
                            entity_id.append(dr_id)
                            entity_value.append(G[k][-1]['@value'])
                            entity_label.append('Goods_Date')
                            rel_to.append(dr_id) # 记得加上rel_to 连接main resource
                            
                        else:
                        # 若是String的值 同样需要创建entity
                            sr_id = 'sR'+ str(stringC) # Character_String sR+ID
                            stringC += 1
                            entity_id.append(sr_id)
                            entity_value.append(G[k][-1]['@value'])
                            entity_label.append('Goods_String')
                            rel_to.append(sr_id) # 记得加上rel_to 连接main resource

In [19]:
print(len(entity_id))
print(len(entity_value))
print(len(entity_label))
print(len(rel_from))
print(len(rel_to))
print(len(rel_type))

880
880
880
777
777
777


In [20]:
entity['Goods:ID(Goods-ID)'] = entity_id
entity['value'] = entity_value
entity[':LABEL'] = entity_label
rel[':START_ID(Goods-ID)'] = rel_from
rel[':END_ID(Goods-ID)'] = rel_to
rel[':TYPE'] = rel_type

In [22]:
entity.to_csv(r'output/goods_entity.csv', encoding='utf-8', index=False)
rel.to_csv(r'output/goods_rel.csv', encoding='utf-8',  index=False)